# Words to Floor Plan (W2FP) Generative adversarial network (GAN)

#### Imports
Packages and libraries used through-out the project.

In [11]:
import json

import tensorflow as tf
import tensorflowjs as tfjs
import matplotlib.pyplot as plt
from tensorflow.keras import Model, layers, losses
from IPython import display

tf.__version__

'2.8.0'

#### Hyper-parametes

In [2]:
MAX_SEQUENCE_LENGTH=128
VOCAB_SIZE=128

#### Text vectorization
Defining text conversion utility, which processes text to be appropriate for neural network input.

In [3]:
int_vectorize_layer = layers.TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)

2022-03-20 08:55:34.286923: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-20 08:55:34.315288: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-20 08:55:34.315466: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-20 08:55:34.315909: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

#### Dataset loading

In [4]:
with open("data/labels.json") as file:
    labels_json = json.load(file)

labels_array = []

for _, value in labels_json.items():
    labels_array.append(value)

labels_tensor = tf.convert_to_tensor(labels_array)
int_vectorize_layer.adapt(labels_tensor)

#### Generator neural network

In [5]:
def create_generator_model() -> Model:
    model = tf.keras.Sequential()

    model.add(int_vectorize_layer)

    # `VOCAB_SIZE + 1` since `0` is used additionally for padding.
    model.add(layers.Embedding(VOCAB_SIZE + 1, 128, input_length=MAX_SEQUENCE_LENGTH, mask_zero=True)),
    model.add(layers.Conv1D(64, 5, padding="same", activation="relu", strides=2))
    model.add(layers.GlobalMaxPooling1D())
    model.add(layers.Dense(8*8*64))

    model.add(layers.Reshape((8, 8, 64)))

    model.add(layers.Conv2DTranspose(512, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(256, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))

    return model

In [6]:
generator = create_generator_model()
generator.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, 128)              0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 128, 128)          16512     
                                                                 
 conv1d (Conv1D)             (None, 64, 64)            41024     
                                                                 
 global_max_pooling1d (Globa  (None, 64)               0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 4096)              266240    
                                                                 
 reshape (Reshape)           (None, 8, 8, 64)          0

In [7]:
noise = int_vectorize_layer("gosho losho posho mosho")
generated_image = generator(noise, training=False)

plt.imshow(generated_image[0, :, :, 0], cmap='gray')

2022-03-20 08:55:35.085301: W tensorflow/core/framework/op_kernel.cc:1722] OP_REQUIRES failed at cast_op.cc:121 : UNIMPLEMENTED: Cast int64 to string is not supported


UnimplementedError: Exception encountered when calling layer "sequential" (type Sequential).

Cast int64 to string is not supported [Op:Cast]

Call arguments received:
  • inputs=tf.Tensor(shape=(128,), dtype=int64)
  • training=False
  • mask=None

#### Discriminator neural network

In [8]:
def create_discriminator_model() -> Model:
    model = tf.keras.Sequential()

    model.add(
        layers.Conv2D(64, (4, 4), strides=(2, 2), padding='same', use_bias=False, input_shape=[64, 64, 1]))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(
        layers.Conv2D(128, (4, 4), strides=(2, 2), padding='same', use_bias=False, input_shape=[64, 32, 32]))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(
        layers.Conv2D(256, (4, 4), strides=(2, 2), padding='same', use_bias=False, input_shape=[128, 16, 16]))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(
        layers.Conv2D(512, (4, 4), strides=(2, 2), padding='same', use_bias=False, input_shape=[256, 8, 8]))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(
        layers.Conv2D(512, (4, 4), strides=(1, 1), padding='same', use_bias=False, input_shape=[512, 4, 4]))

    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model

In [9]:
discriminator = create_discriminator_model()
discriminator.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 64)        1024      
                                                                 
 batch_normalization_4 (Batc  (None, 32, 32, 64)       256       
 hNormalization)                                                 
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 32, 32, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 128)       131072    
                                                                 
 batch_normalization_5 (Batc  (None, 16, 16, 128)      512       
 hNormalization)                                                 
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 16, 16, 128)      

#### Model saving

In [12]:
generator.save('models/tf')
tfjs.converters.save_keras_model(generator, 'models/tfjs')

INFO:tensorflow:Assets written to: models/tf/assets


NotImplementedError: Save or restore weights that is not an instance of `tf.Variable` is not supported in h5, use `save_format='tf'` instead. Received a model or layer TextVectorization with weights [<keras.layers.preprocessing.index_lookup.VocabWeightHandler object at 0x7fbddb94f190>]